<a href="https://colab.research.google.com/github/kethlyncampos/webscraper/blob/main/web_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install requests
!pip install iso8601

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [ ]:
import requests
import pandas as pd
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
import pytz
from datetime import datetime
import iso8601
from dateutil import tz
from google.colab import drive
from selenium.webdriver.common.by import By
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [51]:
yahoo = pd.read_csv('/content/gdrive/MyDrive/news_yahoo.csv')
infomoney = pd.read_csv('/content/gdrive/MyDrive/news_infomoney.csv', lineterminator='\n')

In [52]:
comp_yahoo = []
comp_infomoney = []

for c in yahoo['URL']:
  comp_yahoo.append(c)

for c in infomoney['URL']:
  comp_infomoney.append(c)

In [71]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-notifications")
options.add_argument("start-maximized")
options.add_argument('disable-infobars')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36")
browser = webdriver.Chrome(options=options)

In [54]:
def req_yahoo(url):
  browser.get(url)
  scrollar(browser)
  links = browser.find_elements(By.CSS_SELECTOR, 'a.js-content-viewer.rapidnofollow')
  noticias = []

  for link in links:
    if (('https://br.yahoo.com/news/' not in link.get_attribute('href')) and (link.get_attribute('href') not in comp_yahoo)):
      browser.execute_script("arguments[0].click();", link)
      scrollar(browser)
      WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.TAG_NAME, "h1")))
      temp = BeautifulSoup(browser.page_source, 'html.parser')
      title = temp.find('h1', attrs={'data-test-locator':'headline'}).text
      author = temp.find('span', attrs={'class':'caas-author-byline-collapse'}).text
      collect_date = convert_timezone(datetime.now())
      publication_date = convert_timezone(iso8601.parse_date(temp.find('time')['datetime']))
      description = temp.find('div', attrs={'class':'caas-body'})
      try:
        description.figure.decompose()
        description = description.text
      except:
        description = description.text
      noticias.append(['Yahoo Finanças',title, description, author, publication_date, collect_date, browser.current_url])
      browser.back()
      sleep(3)
  browser.close()
  return noticias

In [73]:
def req_infomoney(url):
  browser.get(url)
  browser.maximize_window()
  alert = browser.switch_to.active_element.click
  infinite = browser.find_element(By.ID, 'infinite-handle')
  browser.execute_script("arguments[0].click();", infinite)
  count = 40
  while(count > 0):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    WebDriverWait(browser, 40).until(EC.presence_of_element_located((By.ID, "infinite-handle")))
    browser.execute_script("arguments[0].click();", infinite)
    newHeight = browser.execute_script("return document.body.scrollHeight")
    lastHeight = newHeight
    count-=1
  page = BeautifulSoup(browser.page_source, 'html.parser')
  browser.close()
  links = page.find_all(class_="hl-title")
  lista_url = []
  for j in links:
    aux = j.find('a')
    if(aux):
      if "stock-pickers" not in aux['href']:
        lista_url.append(aux['href'])
  lista_url = set(lista_url)
  noticias = []
  subtitle = ""
  for k in lista_url:
    if k not in comp_infomoney:
      response = requests.get(k)
      temp = BeautifulSoup(response.content, 'html.parser')
      title = temp.find('h1', attrs={'class': 'page-title-1'})
      if title: # se não houver título, a página é do tipo cotações e possui apenas as informações sobre a empresa
        title = title.text
        subtitle = trata_string(temp.find('p', attrs={'class': 'article-lead'}).text)
        author = temp.find('span', attrs={'class':'author-name'})
        if author:
          author = trata_string(author.find('a').text) # se não houver autor, a página é do tipo guias
          collect_date = convert_timezone(datetime.now())
          publication_date = iso8601.parse_date(temp.find('time', attrs={'class':'entry-date'})['datetime']).strftime('%Y-%m-%d %H:%M:%S')
          description = temp.find('div', attrs={'class':'article-content'})
          try:
            description.figure.decompose()
            description = description.text
          except:
            description = description.text
          description = trata_string(description)
          noticias.append(['Infomoney',title, subtitle, description, author, publication_date, collect_date, k])
  return noticias

In [56]:
def req_investing(url):
  browser2 = webdriver.Chrome(options=options)
  browser2.get(url)
  scrollar(browser2)
  noticias = []
  cont = 0
  while(True):
    links = browser2.find_elements(By.CSS_SELECTOR, '.js-article-item.articleItem .textDiv a')
    for link in links:
      WebDriverWait(browser2, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.js-article-item.articleItem .textDiv a')))
      browser2.execute_script("arguments[0].click();", link)
      scrollar(browser2)
      temp = BeautifulSoup(browser2.page_source, 'html.parser')
      title = temp.find('h1', attrs={'class':'articleHeader'}).text
      time = temp.find('div', attrs={'class':'contentSectionDetails'})
      time = time.find('span').text
      time = re.search('\(([^)]+)', time).group(1)
      publication_date = datetime.strptime(time, '%d.%m.%Y %H:%M')
      publication_date = datetime.strftime(publication_date, '%Y-%m-%d %H:%M:%S')
      collect_date = convert_timezone(datetime.now())
      description = temp.find('div', attrs={'class':'articlePage'})
      try:
        description.find('div',class_='imgCarousel').decompose()
        description = description.text
      except:
        description = description.text
      description = trata_string(description)
      noticias.append(['Investing.com',title, description, publication_date, collect_date, browser2.current_url])
      browser2.back()
      sleep(2)
      scrollar(browser2)
    
    proximo = browser2.find_element(By.CSS_SELECTOR, '.sideDiv.inlineblock.text_align_lang_base_2')
    browser2.execute_script("arguments[0].click();", proximo)
    scrollar(browser2)
    print(browser2.current_url)
    break
  browser2.close()
  return noticias

In [57]:
def scrollar(driver):
  lastHeight = driver.execute_script("return document.body.scrollHeight")
  while True:
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
      sleep(5)
      newHeight = driver.execute_script("return document.body.scrollHeight")
      if newHeight == lastHeight:
          break
      lastHeight = newHeight
  return

In [58]:
def convert_timezone(date_time):
  sp = tz.gettz('America/Sao_Paulo')
  new_date = date_time.replace(tzinfo=pytz.utc)
  new_date = new_date.astimezone(sp)
  new_date = new_date.strftime('%Y-%m-%d %H:%M:%S')
  return new_date

In [59]:
def trata_string(data):
  data = re.sub(r'[\t\n ]+', ' ', data)
  return data

In [60]:
array_noticias = req_yahoo('https://br.yahoo.com/topics/bolsa-de-valores/')

In [61]:
dataset_temp = pd.DataFrame(array_noticias,columns=['Fonte', 'Título', 'Descrição', 'Autor', 'Data/Hora de Publicação', 'Data/Hora de Coleta', 'URL'])

In [63]:
if len(dataset_temp) > 0:
  dataset_temp.drop_duplicates(subset=['Descrição'], inplace = True)
  dataset_temp.reset_index(drop = True, inplace = True)
  yahoo = pd.concat([dataset_temp,yahoo],ignore_index=True)
  yahoo.drop_duplicates(subset=['Descrição'], inplace = True)
  yahoo.drop(yahoo.filter(regex="Unname"),axis=1, inplace=True)
  yahoo = yahoo.reset_index(drop=True)
  yahoo.to_csv('/content/gdrive/MyDrive/news_yahoo.csv', index=False)

In [74]:
noticias_info = req_infomoney("https://www.infomoney.com.br/mercados/")

In [75]:
dataset_temp = pd.DataFrame(noticias_info,columns=['Fonte', 'Título', 'Subtítulo','Descrição', 'Autor', 'Data/Hora de Publicação', 'Data/Hora de Coleta', 'URL'])

In [76]:
if len(dataset_temp) > 0:
  dataset_temp.drop_duplicates(subset=['Descrição'], inplace = True)
  dataset_temp.reset_index(drop = True, inplace = True)
  infomoney = pd.concat([dataset_temp, infomoney])
  infomoney.drop_duplicates(subset=['Descrição'], inplace = True)
  infomoney.drop(infomoney.filter(regex="Unname"),axis=1, inplace=True)
  infomoney = infomoney.reset_index(drop=True)
  infomoney.to_csv('/content/gdrive/MyDrive/news_infomoney.csv', index=False)

In [ ]:
page2 = req_investing('https://br.investing.com/news/stock-market-news')

In [ ]:
dataset_temp = pd.DataFrame(array_noticias,columns=['Fonte', 'Título', 'Descrição', 'Data/Hora de Publicação', 'Data/Hora de Coleta', 'URL'])